# Make super clean datafile!

respecting the block structure and the fact that the first history columns within a block must be nans. 

In [1]:
import os, sys, glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import operator

# for frequentist model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures

# settings for more convenient df inspection
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [2]:
# i need a datafile that also has block type and difficulty level
df_all = pd.read_csv('C:\\Users\\annae\\Dropbox\\PhD\\click_gamified+\\ONLINE_EXPERIMENT\\FINAL_DATA\\combined_data_all_task_versions.csv')
subjects = list(df_all['Participant Public ID'].dropna().unique())

pdi_res = pd.read_csv('C:\\Users\\annae\\Dropbox\\PhD\\click_gamified+\\ONLINE_EXPERIMENT\\FINAL_DATA\\pdi_results.csv')
caps_res = pd.read_csv('C:\\Users\\annae\\Dropbox\\PhD\\click_gamified+\\ONLINE_EXPERIMENT\\FINAL_DATA\\caps_results.csv')

C:\Users\annae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# EXCLUSIONS
responses = ['Right', 'Left', '2500']
exclusions, included = [], []

for i in range(1, len(subjects)): 
    # exclude incomplete datasets
    df_sbj = df_all[df_all['Participant Public ID'] == subjects[i]]
    df_sbj_rel = df_sbj[df_sbj['Response'].isin(responses)]
    
    if df_sbj_rel.shape[0] != 385: 
        exclusions.append(subjects[i])
        
    # exclude performance based criterion
    corr = df_sbj_rel['Correct'].value_counts()[1.0]
    wron = df_sbj_rel['Correct'].value_counts()[0.0]
    
    perc = corr / (corr+wron)*100
    
    if perc > 90: 
        exclusions.append(subjects[i])
    elif perc < 60: 
        exclusions.append(subjects[i])
        
    # exclude attention check missers
    attention = df_sbj[df_sbj['Zone Type']=='response_keyboard_single']
    attention_check_correct = attention.Correct.sum()
    
    if attention_check_correct < 20: 
        exclusions.append(subjects[i])
        
    # exclude ppl who didn't make it to the questionnaires
 
    if caps_res[caps_res['Subject_ID']==subjects[i]].shape == (0,6): 
        exclusions.append(subjects[i])
    if pdi_res[pdi_res['Subject_ID']==subjects[i]].shape==(0,6):
        exclusions.append(subjects[i])
    
    if subjects[i] not in exclusions: 
        included.append(df_sbj_rel)
        
df_inc = pd.concat(included)
print(len(included))

113


In [4]:
cols = ['sbj_id','trial','block_type','block','stimulus','target','response','cue','caps','pdi']

df_reg = pd.DataFrame(columns=cols)


for df_sbj in included: 
    
    block_ends = [49*i for i in range(1,9)]
    trial_ct = 1
    block_ct = 1
    sbj_id = list(df_sbj['Participant Public ID'])[0]
    
    for index, row in df_sbj.iterrows(): 
        
        if trial_ct in block_ends: 
            block_ct += 1
            
        # block_type
        if row['meta_typeblock'] == 'rep': 
            block_type = 0
        elif row['meta_typeblock'] == 'neut': 
            block_type = 1
        
        # cue
        if row['cue'] == 'cue_right.PNG': 
            cue = 0 # right cue
        elif row['cue'] == 'cue_left.PNG': 
            cue = 1 # left cue
        else: 
            print('sth wrong')
        
        # stim, target, response
        stim = row['meta_difflevel']/10 # in int dtype
        target = row['ANSWER'] # correct answer
        resp = row['Response'] # given answer

        if target == 'Right': 
            target = 0
        elif target == 'Left': 
            target = 1

        if resp == 'Right': 
            resp = 0
        elif resp == 'Left':
            resp = 1
        elif resp == '2500': # time-outs: I classify them as incorrect for now
            
            if target == 1: 
                resp = 0
            elif target == 0: 
                resp = 1
                
        pdi = pdi_res[pdi_res['Subject_ID'] == sbj_id]['sum score'].values[0]
        caps = caps_res[caps_res['Subject_ID'] == sbj_id]['sum score'].values[0]
                
        row = {'sbj_id':sbj_id,'trial':trial_ct,'block_type':block_type,'block':block_ct,
               'stimulus':stim,'target':target,'response':resp,'cue':cue, 'pdi':pdi, 'caps':caps}
        
        df_reg = df_reg.append(row, ignore_index=True)
        
        trial_ct += 1
        

In [5]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
hists = ['resp_%i'%i for i in range(1,2)]

# make history columns with NaNs
def make_nan_col(df, name): 
    df[name] = np.nan


# collect previous k response and make them NaN at beginning of blocks

def collect_history(df, k):
 
    for index, row in df.iterrows():

        prev = df[df.index==index-k]['response'].values.astype(int)

        if prev.shape == (0,): 
            prev = np.nan
            
        elif df.at[index, 'block'] != df.at[index-1, 'block']: 
            prev = np.nan
            
        else: 
            prev = prev[0]
            
        target_col = str('resp_%i'%k)
        df.at[index, target_col] = prev


In [6]:
for name in hists: 
    make_nan_col(df_reg, name)

# reset index
df = df_reg.reset_index(drop=True)

In [7]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
for k in range(1,2): 
    collect_history(df, k)

In [8]:
df.head()

,sbj_id,trial,block_type,block,stimulus,target,response,cue,caps,pdi,resp_1
0,5e6d3324f14d262fced5bbfe,1,0,1,0.3,1,1,0,5,5,NaN
1,5e6d3324f14d262fced5bbfe,2,0,1,0.2,1,1,1,5,5,1.0
2,5e6d3324f14d262fced5bbfe,3,0,1,0.6,1,0,1,5,5,1.0
3,5e6d3324f14d262fced5bbfe,4,0,1,0.2,1,1,1,5,5,0.0
4,5e6d3324f14d262fced5bbfe,5,0,1,0.6,1,1,1,5,5,1.0


In [9]:
def make_nans_history(df, k):
    
    target_col = 'resp_%i'%k
    resps = [0.0, 1.0]
    
    indexes = []
    
    for index, row in df.iterrows():
    
        if df.at[index, target_col] not in resps:
            indexes.append(index)
            
    for i in indexes: 
    
        if i > 7:
            
            for j in range(1,k): 

                df.at[i+j, target_col] = np.nan
              
    return indexes

In [1]:
# only for longer term history
#for i in range(2,8):
#    make_nans_history(df,i)
#df.shape[0]/len(included)

In [ ]:
# write to csv - uncomment if not needed. 
df.to_csv('final_clean_exclusions.csv')

# stimulus history!


In [ ]:
df = pd.read_csv('final_clean_exclusions.csv')

### add previous stimuli

In [12]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
stims = ['stim_%i'%i for i in range(1,2)]

for stim in stims: 
    make_nan_col(df, stim)
    
df = df.reset_index(drop=True)

# collect stimulus history and make it nan at block start
def collect_history(df, k):
 
    for index, row in df.iterrows():

        prev = df[df.index==index-k]['target'].values.astype(int)
        
        if prev.shape == (0,): 
            prev = np.nan
            
        elif df.at[index, 'block'] != df.at[index-1, 'block']: 
            prev = np.nan
            
        else: 
            prev = prev[0]
            
        target_col = str('stim_%i'%k)
        df.at[index, target_col] = prev


In [13]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
for k in range(1,2): 
    collect_history(df, k)

In [14]:
def make_nans_history(df, k):
    
    target_col = 'stim_%i'%k
    stim = [0.0, 1.0]
    
    indexes = []
    
    for index, row in df.iterrows():
    
        if df.at[index, target_col] not in stim:
            indexes.append(index)
            
    for i in indexes: 
    
        if i > 7:
            
            for j in range(1,k): 

                df.at[i+j, target_col] = np.nan
              
    return indexes

In [ ]:
# for full datafile as used in the incremental model comparison, change to range(1,8)
for k in range(1,2): 
    make_nans_history(df, k)

In [ ]:
df.to_csv('final_clean_stimulus.csv')

In [ ]:
df.head()

In [18]:
# read super clean datafile that contains stimulus history as well.
#os.chdir('C:\\Users\\annae\\Dropbox\\PhD\\CODE\\data_preprocessed')
#df = pd.read_csv('final_clean_stimulus.csv')
# drop random unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# add PPS score, sum of z-standardized pdi and caps scores
pps_measures = ['pdi', 'caps']

for measure in pps_measures: 
    col_zscore = measure + '_zscore'
    df[col_zscore] = (df[measure] - df[measure].mean())/df[measure].std(ddof=0)

df['PPS_z'] = df['pdi_zscore'] + df['caps_zscore']


# z-standardise columns
cols = list(df.columns)
[cols.remove(item) for item in ['sbj_id', 'trial', 'block', 'caps_zscore', 'pdi_zscore', 'PPS_z']]

for col in cols: 
    col_z = col + '_z'
    df[col_z] = (df[col]- df[col].mean())/df[col].std(ddof=0)
    
df = df.dropna()

df.to_csv('1_model1_auditory.csv')

In [19]:
df.head()

,sbj_id,trial,block_type,block,stimulus,target,response,cue,caps,pdi,resp_1,stim_1,pdi_zscore,caps_zscore,PPS_z,block_type_z,stimulus_z,target_z,response_z,cue_z,caps_z,pdi_z,resp_1_z,stim_1_z,block_type_z_z,stimulus_z_z,target_z_z,response_z_z,cue_z_z,caps_z_z,pdi_z_z,resp_1_z_z,stim_1_z_z
1,5e6d3324f14d262fced5bbfe,2,0,1,0.2,1,1,1,5,5,1.0,1.0,-0.644205,-0.37994,-1.02414,-0.999226,-0.882553,0.995106,1.00181,0.963416,-0.37994,-0.644205,1.002797,0.993006,-0.999226,-0.882553,0.995106,1.00181,0.963416,-0.37994,-0.644205,1.002797,0.993006
2,5e6d3324f14d262fced5bbfe,3,0,1,0.6,1,0,1,5,5,1.0,1.0,-0.644205,-0.37994,-1.02414,-0.999226,1.458884,0.995106,-0.998194,0.963416,-0.37994,-0.644205,1.002797,0.993006,-0.999226,1.458884,0.995106,-0.998194,0.963416,-0.37994,-0.644205,1.002797,0.993006
3,5e6d3324f14d262fced5bbfe,4,0,1,0.2,1,1,1,5,5,0.0,1.0,-0.644205,-0.37994,-1.02414,-0.999226,-0.882553,0.995106,1.00181,0.963416,-0.37994,-0.644205,-0.997211,0.993006,-0.999226,-0.882553,0.995106,1.00181,0.963416,-0.37994,-0.644205,-0.997211,0.993006
4,5e6d3324f14d262fced5bbfe,5,0,1,0.6,1,1,1,5,5,1.0,1.0,-0.644205,-0.37994,-1.02414,-0.999226,1.458884,0.995106,1.00181,0.963416,-0.37994,-0.644205,1.002797,0.993006,-0.999226,1.458884,0.995106,1.00181,0.963416,-0.37994,-0.644205,1.002797,0.993006
5,5e6d3324f14d262fced5bbfe,6,0,1,0.3,1,1,1,5,5,1.0,1.0,-0.644205,-0.37994,-1.02414,-0.999226,-0.297193,0.995106,1.00181,0.963416,-0.37994,-0.644205,1.002797,0.993006,-0.999226,-0.297193,0.995106,1.00181,0.963416,-0.37994,-0.644205,1.002797,0.993006


In [28]:
list(df['sbj_id'].unique())

['5e6d3324f14d262fced5bbfe',
 '5f3a851741c4f9694eafe24a',
 '5be32a4a36b93f0001d9b926',
 '5f4432700cbdc61b7ab46082',
 '5f577cc710fc57212ddfdfe2',
 '5d989cb14658d5001883ff18',
 '5fa17937504cb3282bf86517',
 '5ef658dc548fce12fda46af8',
 '5ecd21dec04cca02c9032485',
 '5d372d6330b7be00166f6bce',
 '5f36ba1b5efc210b15d8a1fc',
 '5ee258718b3a1d0f6e6bb067',
 '5f2aee4b1a903c446deb0083',
 '5f85901946ca0a000a802cf2',
 '5ec3b9e83f187802a6aa041e',
 '5e9f31e7a7eac50a41be51b7',
 '5f593ffc7fa1481697b363e5',
 '5f50d630390521264e606772',
 '5d00bb3b51669b000136a986',
 '5f53216409e6d06275ef0545',
 '5f44f6de7f24ec0354450af6',
 '5c1246ea72a5f1000117134b',
 '5ee758de233cdb01d6a03f3d',
 '5f2c3cb1c58e050be52d9161',
 '5d5c24635c2543001625d50f',
 '5d90d51053e62800193f9a94',
 '5e36b149a617e5637898ab7e',
 '5f5101455eceb72b2e47ef13',
 '5d68c8aa40524c00189e8ac2',
 '5f89e17a57925103ac8601df',
 '5f315af6d221ae0e7843dc13',
 '5eaadc0a7adeb404eea9c3c0',
 '5eb06e62d2a3d710cbcf1850',
 '5f3b1f2078bf3c22f24a262c',
 '5f2a9ae7e01a